In [1]:
from pathlib import Path
import pandas as pd

In [2]:
def grouped_results(merged_df, agg, target_column, join_symbol="±"):
    formating = lambda x: "{:.2f}".format(x)
    grouped_mean_formatted = merged_df.groupby(agg)[target_column].mean().map(formating)
    grouped_std_formatted = merged_df.groupby(agg)[target_column].std().map(formating)
    result_df = pd.concat([grouped_mean_formatted, grouped_std_formatted], axis=1)
    result_df.columns = ["Mean", "Std"]

    # Combine mean and std columns to display mean ± std
    result_df["Mean ± Std"] = result_df["Mean"] + f"{join_symbol}" + result_df["Std"]

    result_df["Mean ± Std"] = result_df["Mean ± Std"].map(lambda x: f"${x}$")
    return result_df


def compute_diffs(merged_df, merge_on=None, metric="avg_acc_tag"):
    if not merge_on:
        merge_on = ["num_tasks", "approach", "dataset", "seed"]
    reg_false_df = merged_df[merged_df["reg_layers"] == False]
    reg_true_df = merged_df[merged_df["reg_layers"] == True]

    # Merge the two dataframes based on 'num_classes', 'approach', 'dataset', and 'seed'
    merged_reg_diff = pd.merge(
        reg_false_df,
        reg_true_df,
        on=merge_on,
        suffixes=("_false", "_true"),
        how="inner",
    )
    merged_reg_diff[f"{metric}_diff"] = (
        merged_reg_diff[f"{metric}_true"] - merged_reg_diff[f"{metric}_false"]
    )
    return merged_reg_diff


def get_merged_df(paths):
    dfs = [pd.read_csv(path) for path in paths]

    merged_df = pd.concat(dfs, axis=0)
    return merged_df

# Big tables

In [8]:
root = Path("csvs_new")
convnext = root / "resnet34_skips"
paths = list(convnext.glob("imagenet*.csv"))


merged_df = get_merged_df(paths)

In [9]:
agg = ["num_tasks", "dataset", "approach", "reg_layers"]
grouped_results(merged_df, agg, "avg_acc_tag", join_symbol="\pm")

Mean   Std  \
num_tasks dataset                approach   reg_layers                
10        imagenet_subset_kaggle ewc        False       13.73  0.20   
                                            True        16.67  0.74   
                                 finetuning False       11.52  0.14   
                                            True        12.42  0.90   
                                 lwf        False       35.47  1.22   
                                            True        37.97  0.85   
                                 replay     False       38.59  0.88   
                                            True        41.29  0.06   
20        imagenet_subset_kaggle ewc        False        7.01  0.15   
                                            True         5.28  0.07   
                                 finetuning False        6.08  0.30   
                                            True         5.26  0.31   
                                 lwf        False       19.61  0.46   
                                            True        21.17  0.40   
                                 replay     False       35.00  2.15   
                                            True        37.57  0.42   

                                                            Mean ± Std  
num_tasks dataset                approach   reg_layers                  
10        imagenet_subset_kaggle ewc        False       $13.73\pm0.20$  
                                            True        $16.67\pm0.74$  
                                 finetuning False       $11.52\pm0.14$  
                                            True        $12.42\pm0.90$  
                                 lwf        False       $35.47\pm1.22$  
                                            True        $37.97\pm0.85$  
                                 replay     False       $38.59\pm0.88$  
                                            True        $41.29\pm0.06$  
20        imagenet_subset_kaggle ewc        False        $7.01\pm0.15$  
                                            True         $5.28\pm0.07$  
                                 finetuning False        $6.08\pm0.30$  
                                            True         $5.26\pm0.31$  
                                 lwf        False       $19.61\pm0.46$  
                                            True        $21.17\pm0.40$  
                                 replay     False       $35.00\pm2.15$  
                                            True        $37.57\pm0.42$

In [11]:
agg = ["num_tasks", "dataset", "approach"]
merged_reg_diff = compute_diffs(merged_df)
grouped_results(merged_reg_diff, agg, "avg_acc_tag_diff", join_symbol="\pm")

Mean   Std      Mean ± Std
num_tasks dataset                approach                               
10        imagenet_subset_kaggle ewc          2.94  0.92   $2.94\pm0.92$
                                 finetuning   0.90  1.04   $0.90\pm1.04$
                                 lwf          2.50  1.84   $2.50\pm1.84$
                                 replay       2.69  0.85   $2.69\pm0.85$
20        imagenet_subset_kaggle ewc         -1.73  0.11  $-1.73\pm0.11$
                                 finetuning  -0.82  0.20  $-0.82\pm0.20$
                                 lwf          1.56  0.76   $1.56\pm0.76$
                                 replay       2.57  1.82   $2.57\pm1.82$

# Big task

In [18]:
root = Path("csvs")
convnext = root / "big_task"
paths = list(convnext.glob("*.csv"))


merged_df = get_merged_df(paths)

In [19]:
merged_df

,reg_layers,real_name,wavg_acc_tag,misc.seed,data.datasets
0,True,replay,35.79,1,cifar100_fixed
1,True,lwf,50.79,1,cifar100_fixed
2,True,ewc,38.76,1,cifar100_fixed
3,True,finetuning,28.29,1,cifar100_fixed
4,False,lwf,43.58,1,cifar100_fixed
5,False,ewc,27.89,1,cifar100_fixed
6,False,replay,29.74,1,cifar100_fixed
7,False,finetuning,22.14,1,cifar100_fixed
0,True,replay,35.87,0,cifar100_fixed
1,True,lwf,54.09,0,cifar100_fixed


In [25]:
agg = ["real_name", "reg_layers"]
grouped_results(merged_df, agg, "wavg_acc_tag", join_symbol="\pm ")

Mean   Std       Mean ± Std
real_name  reg_layers                              
ewc        False       27.97  2.40  $27.97\pm 2.40$
           True        38.10  1.61  $38.10\pm 1.61$
finetuning False       21.92  1.43  $21.92\pm 1.43$
           True        28.16  1.27  $28.16\pm 1.27$
lwf        False       43.58  1.25  $43.58\pm 1.25$
           True        52.13  1.73  $52.13\pm 1.73$
replay     False       29.23  0.48  $29.23\pm 0.48$
           True        35.32  0.88  $35.32\pm 0.88$

In [24]:
agg = ["real_name"]
merged_reg_diff = compute_diffs(
    merged_df,
    merge_on=["real_name", "misc.seed", "data.datasets"],
    metric="wavg_acc_tag",
)
grouped_results(merged_reg_diff, agg, "wavg_acc_tag_diff", join_symbol="\pm ")

,Mean,Std,Mean ± Std
real_name,,,
ewc,10.13,1.10,$10.13\pm 1.10$
finetuning,6.25,0.17,$6.25\pm 0.17$
lwf,8.55,1.16,$8.55\pm 1.16$
replay,6.10,0.60,$6.10\pm 0.60$


In [ ]:
    $21.92$ \tiny{$\pm 1.43$}
    $28.16$ \tiny{$\pm 1.27$}
    $6.25$ \tiny{$\pm 0.17$}
    $29.23$ \tiny{$\pm 0.48$}
    $35.32$ \tiny{$\pm 0.88$}
    $6.10$ \tiny{$\pm 0.60$}
    $27.97$ \tiny{$\pm 2.40$}
    $38.10$ \tiny{$\pm 1.61$}
    $10.13$ \tiny{$\pm 1.10$}
    $43.58$ \tiny{$\pm 1.25$}
    $52.13$ \tiny{$\pm 1.73$}
    $8.55$ \tiny{$\pm 1.16$}
